In [38]:
import os
import numpy as np
import pandas as pd
import math
from pyomo.environ import *
from collections import defaultdict
from pyomo.opt import SolverFactory
from datetime import timedelta
from datetime import datetime
import random

import shutil
import subprocess
random.seed(111)

In [39]:
now = datetime.now()
now = now.strftime("%m%d_%H_%M")
os.mkdir(now)

raw = pd.read_csv('../../init_data.csv', header=0, index_col=0)


#general param
states_name = raw['State']
first_date = '2020/03/25'

t = 7
T = 14
P = 10
S = 51
P = 10

lbd = 0.01
SNS_stock = 12000
init_ratio = 0.6
flow_bound_ratio = 0.20
stock_bound_ratio = 0.5

#write to file
with open('temp.txt', 'w') as f:
    f.write('now=\'{}\'\ndate=\'{}\''.format(now, first_date))
    
with open(now + '/config.txt', 'w') as f:
    f.write('lbd={} \nSNS_stock={} \ninit_ratio={} \nflow_bound_ratio={} \nstock_bound_ratio={}\n'.format(lbd, SNS_stock, init_ratio, flow_bound_ratio,stock_bound_ratio))

In [40]:
prev_date = pd.to_datetime(first_date) - timedelta(t)
data = raw.rename(columns={'Available capacity': "stock_self"})
data['stock_self'] = round(init_ratio * data['stock_self'])
data['stock_nb'] = None
for i in range(S):
    data.loc[i, 'stock_nb'] = str([0 for _ in eval(data.loc[i, 'nb'])])
data['stock_sns'] = [0 for _ in range(S)]
data.to_csv(now + '/' + prev_date.strftime('%m%d') + '.csv')

In [41]:
idx = 0
flow_mapping = {}
flow_mapping_rev = {}
out_flow = defaultdict(list)
in_flow = defaultdict(list)

for i in range(51):
    nbs = eval(data.iloc[i, 3])
    out_flow[i] = nbs
    for n in nbs:
        flow_mapping[(i, n)] = idx
        flow_mapping_rev[idx] = (i, n)
        in_flow[n].append(i)
        idx += 1

idx_nb = idx
in_flow[51] = []
out_flow[51] = list(range(51))
for n in range(51):
    flow_mapping[(51, n)] = idx
    flow_mapping_rev[idx] = (51, n)
    idx += 1

In [42]:

def sto_file_generator(date, sto):
    #open the sto file
    prob_name = 'GRA_{}'.format(date.strftime("%m%d"))
    sto_file = open(sto, 'w')
    
    sto_file.write('STOCH ' + prob_name + ' \n')
    sto_file.write('BLOCKS DISCRETE REPLACE\n')
    probability = 1 / 3
    
    
    for j in range(S):
        demand_state = pd.read_csv('../../IHME/' + date.strftime("%m%d") + '/' + data.loc[j, 'State'] + '.csv', header=0, index_col=0)
        d = [demand_state['InvVen_lower'], demand_state['InvVen_mean'], demand_state['InvVen_upper']]

        paths = [[], [], []]
        paths[0] = list(d[1] + 0.5 * (d[2] - d[1]))[0:7] + list(d[1] + 0.75 * (d[2] - d[1]))[7:14] \
            + list(d[1] + 0.5 * (d[2] - d[1]))[7:14] + list(d[1] + 0.25 * (d[2] - d[1]))[7:14]
        paths[1] = list(d[1])[0:7] + list(d[1] + 0.25 * (d[2] - d[1]))[7: 14] \
            + list(d[1])[7:14] + list(d[1] + 0.25 * (d[0] - d[1]))[7: 14]
        paths[2] = list(d[1] + 0.5 * (d[0] - d[1]))[0:7] + list(d[1] + 0.25 * (d[0] - d[1]))[7:14] \
            + list(d[1] + 0.5 * (d[0] - d[1]))[7:14] + list(d[1] +0.75 * (d[0] - d[1]))[7:14] 
        

        for k in range(3):
            sto_file.write(' BL BLOCK{} PERIOD2 {}\n'.format(j, probability))

            path = {}
            for time in range(1, t + 1):
                n = 'c_e_dummy_d({}_{})_'.format(j, time)
                path[n] =  paths[k].pop(0)
            for time in range(t + 1, T + 1):
                for sub_k in range(3):
                    n = 'c_e_dummy_d_k({}_{}_{})_'.format(j, time, sub_k)
                    path[n] =  paths[k].pop(0)

            path_sorted = sorted(path.items())
            for i in path_sorted:
                sto_file.write('\t RHS {} {}\n'.format(i[0], i[1]))
  
    sto_file.write('ENDATA \n')
    sto_file.close()

In [43]:
# subprocess.run(['./SD_xcode', 'GRA_{}'.format(date.strftime("%m%d")),'./spInput/', './spOutput'], cwd='/Users/jjxu/Desktop/covid_system/SD/')

In [44]:
def analysis_results(now, date):
    prev_date = date - timedelta(7)
    data = pd.read_csv('{}/{}.csv'.format(now, prev_date.strftime("%m%d")), header=0, index_col=0)
    sol = pd.read_csv('{}/res/GRA_{}/sol.csv'.format(now, date.strftime("%m%d")), header=None, index_col=None)
    new_stock_self = [0 for _ in range(52)]
    new_flow = defaultdict(int)
    new_stock_sns = [0 for _ in range(51)]
    for i in range(sol.shape[0]):
        name = sol.iloc[i, 0].split('(')
        if name[0] == 's0':
            new_stock_self[int(name[1].split(')')[0])] = int(round(sol.iloc[i, 1]))
        if name[0] == 's0_nb':
            cnt = int(name[1].split(')')[0])
            f, t = flow_mapping_rev[cnt] 
            if f == 51:
                new_stock_sns[t] = int(round(sol.iloc[i, 1]))    
            else:
                new_flow[flow_mapping_rev[cnt]] = int(round(sol.iloc[i, 1]))
            
            
    data['stock_self'] = new_stock_self[:-1]
    data['stock_sns'] = new_stock_sns
    for i in range(51):
        l1 = eval(data.loc[i, 'nb_idx'])
        l2 = [new_flow[j, i] for j in l1]
        data.loc[i, 'stock_nb'] = str(l2)
        
    data.to_csv('{}/{}.csv'.format(now, date.strftime("%m%d")))
    

In [45]:
#generate SPMS model
#solve with SD
#generate new initial data file
def solve_model(now, date):
    #now: string format
    #date: datetime format
    #generate SMPS
    os.system("python -m pyomo.pysp.convert.smps -m GRA.py --symbolic-solver-labels --basename GRA_{} --output-directory {}/GRA_{}".format(date.strftime("%m%d"), now, date.strftime("%m%d")))
    sto = "{}/GRA_{name}/GRA_{name}.sto".format(now, name=date.strftime("%m%d"))
    if os.path.exists(sto):
        os.remove(sto)
    sto_file_generator(date, sto)
    
    #move sd input file
    source = "{}/GRA_{name}".format(now, name=date.strftime("%m%d"))
    destination = "../../SD/spInput/"
    input_file = "../../SD/spInput/GRA_{}".format(date.strftime("%m%d"))
    if os.path.exists(input_file):
        os.remove(input_file)
    shutil.move(source, destination)
    
    #run SD
    subprocess.run(['./SD_xcode', 'GRA_{}'.format(date.strftime("%m%d")),'./spInput/', './spOutput'], cwd='/Users/jjxu/Desktop/covid_system/SD/')
    source = "../../SD/spOutputtwoSD/GRA_{}".format(date.strftime("%m%d"))
    destination = "{}/res/GRA_{}".format(now, date.strftime("%m%d"))
    shutil.move(source, destination)
    
    #analysis current solution and generate new init data
    analysis_results(now, date)

In [46]:
for i in range(P):
    date = pd.to_datetime(first_date) + timedelta(i * t)
    with open('temp.txt', 'w') as f:
        f.write('now=\'{}\'\ndate=\'{}\''.format(now, date.strftime('%Y/%m/%d')))
        
    solve_model(now, date)  
    print('Finish {}'.format(date.strftime('%m%d')))

Finish 0325
Finish 0401
Finish 0408
Finish 0415
Finish 0422
Finish 0429
Finish 0506
Finish 0513
Finish 0520
Finish 0527
